<a href="https://colab.research.google.com/github/takemo4/kids_anagram_app/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U google-generativeai

In [ ]:
# 1,2 Flask, pyngrokのインストール
!pip install flask pyngrok
#  必要なモジュールのimport
import os
# flaskからFlaskクラスをインポート
from flask import Flask
# pyngrokからngrokモジュールとconfモジュールをインポート
from pyngrok import ngrok, conf
# ngrokトークンを設定
conf.get_default().auth_token = "" #べた張り対策


In [ ]:
#Googleドライブにマウント
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# GeminiAPIの設定
API_KEY = ""  # GeminiAPIのキーを入力

import google.generativeai as genai
import random
genai.configure(api_key=API_KEY)

#英語をひとつ生成
def make_word():
    # Gemini API に対するプロンプト（ランダム性を持たせるための工夫が必要）
    X = random.randint(1, 100)
    prompt = f"generate one random English word between 3 and 10 characters.Think {X} words.and tel me only one word"
    # APIリクエスト用のデータ
    try:
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        word = response.text
        return word
    except requests.exceptions.RequestException as e:
        print(f"APIエラー: {e}")
        return None
    except (IndexError, ValueError) as e:
        print(f"解析エラー: {e}")
        return None

#ランダムに並び替えた文字を3つ作成。アナグラム、選択肢としてリターン
def make_anagram(word, count=3):
    word_set = set()  # 重複を防ぐためのセット
    while len(word_set) < count:
        word_list = list(word)
        random.shuffle(word_list)
        anagram = ''.join(word_list)
        if anagram != word:
            word_set.add(anagram)
    return list(word_set)


#どっちの処理も合わせる
def fetch_anagram():
    answer_list = []
    word = (make_word().replace("\n", "")).replace("*","")
    original = word
    words_anagram = (make_anagram(word))
    choices = [words_anagram[1],words_anagram[2],original]
    random.shuffle(choices)
    return {
    "anagram": words_anagram[0],
    "original": original,
    "choices": choices
    }
print(make_word())
print(fetch_anagram())

Fuchsia

{'anagram': 'exyngO', 'original': 'Oxygen', 'choices': ['Oxygen', 'eOnygx', 'nyOxge']}


In [ ]:
import requests
from flask import Flask, render_template, request,jsonify

app = Flask(__name__,template_folder='/content/drive/MyDrive/kids_anagram/templates')

@app.route("/")
def home():
    return render_template("index.html")

In [ ]:
@app.get("/quiz")
def move_quiz():

   result = fetch_anagram()
   if result:
        return render_template("quiz.html",
                               anagram=result["anagram"],
                               options=result["choices"],
                               correct_answer=result["original"])
   return "APIエラー: 問題を生成できませんでした。", 500

In [ ]:
if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print(f"ngrok URL: {public_url}")
    app.run(port=5000)

ngrok URL: NgrokTunnel: "https://44e8-35-221-183-227.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Nov/2024 10:28:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2024 10:28:50] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2024 10:29:45] "GET / HTTP/1.1" 200 -
